# Genex Research Agents (Lite) — Single-Model Therapy Research + Parent Summary (ADK Notebook)

## What this notebook is
This notebook is a **lightweight Genex research agent** built with **Google ADK (Agent Development Kit)**. It provides a **simplified version of the research pipeline**, using a **single LLM (GPT)** instead of parallel multi-model research.

It is designed for:
- faster iteration
- lower API cost
- easier debugging
- demos where reliability matters more than model diversity

> This notebook produces educational, parent-support content and **does not replace medical advice**.

---

## Inputs (what you must provide)
### 1) OpenAI API key
The notebook uses GPT via `OpenAI(...)`.
- Set `OPENAI_API_KEY` in your environment or `.env`.
- Keys are loaded with `python-dotenv`.

### 2) Parent prompts (runtime)
You interact through:
- `await ask("My child's name is ..., age ..., diagnosis ...")`
- `await ask("What therapies help with ...?")`
- `await ask("Give me more activities for ...")`

---

## What it does (process overview)
### A) Child profile storage
- A **Profile Agent** extracts **name / age / diagnosis** from parent input.
- The profile is stored in ADK **session state** via `save_child_profile`.
- Follow-up questions reuse this stored profile automatically.

### B) Research + summarization (single agent)
- A single **Research Agent**:
  - retrieves the stored profile
  - generates age- and diagnosis-aware therapy recommendations
  - responds directly with a structured, parent-friendly answer
- No parallel agents or aggregator are used in this version.

This makes the execution path:
**Profile → Research → Final Answer**

---

## Outputs (what you get)
### Console output
- The helper `ask()` prints the final GPT-generated response.
- There is no separate aggregation step; the research agent’s response *is* the final output.

### Session state artifacts (in-memory)
Using `InMemorySessionService`, the following are stored during the session:
- `child_profile` (name, age, diagnosis)

State resets if the kernel restarts or the session id changes.

---

## How to run
1) Run all cells top-to-bottom to initialize tools, agents, and runner.
2) Start by sending a message with the child’s profile (required once per session).
3) Ask follow-up therapy questions freely in the same session.

---

## When to use this notebook
- Local experimentation
- UI demos
- Early-stage prototyping
- Situations where one strong model is sufficient

## Limitations / known constraints
- No cross-model validation or diversity of viewpoints.
- Output quality depends entirely on GPT behavior and prompt quality.
- Not a clinical or diagnostic system.


In [ ]:
# ==============================
# 0. Imports & basic setup
# ==============================
import os
from typing import Any, Dict

from dotenv import load_dotenv

from google.adk.agents import LlmAgent, SequentialAgent, ParallelAgent
from google.adk.apps.app import App
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import FunctionTool
from google.adk.tools.tool_context import ToolContext
from google.adk.models.lite_llm import LiteLlm

# Direct API clients used *inside tools*
from openai import OpenAI
from anthropic import Anthropic

# --- Load keys ---
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")

openai_client = OpenAI(api_key=OPENAI_API_KEY) if OPENAI_API_KEY else None
anthropic_client = Anthropic(api_key=ANTHROPIC_API_KEY) if ANTHROPIC_API_KEY else None

print("API keys (first 5 chars):")
print("  OpenAI:", (OPENAI_API_KEY or "MISSING")[:5])
print("  Anthropic:", (ANTHROPIC_API_KEY or "MISSING")[:5])

# One session service for this notebook
session_service = InMemorySessionService()

### Child profile tool 

In [2]:
# ==============================
# 1. Child profile tools
# ==============================

def save_child_profile(
    tool_context: ToolContext,
    name: str,
    age_years: int,
    diagnosis: str,
) -> Dict[str, Any]:
    """
    Save the child's profile (name, age_years, diagnosis) into session state.
    """
    clean_name = name.strip()
    clean_diagnosis = diagnosis.strip()

    tool_context.state["user:child:name"] = clean_name
    tool_context.state["user:child:age_years"] = int(age_years)
    tool_context.state["user:child:diagnosis"] = clean_diagnosis

    return {
        "status": "success",
        "message": f"Stored child profile: {clean_name}, {age_years} years old, diagnosis: {clean_diagnosis}.",
        "name": clean_name,
        "age_years": int(age_years),
        "diagnosis": clean_diagnosis,
    }


def retrieve_child_profile(tool_context: ToolContext) -> Dict[str, Any]:
    """
    Retrieve child profile from session state.
    """
    name = tool_context.state.get("user:child:name")
    age_years = tool_context.state.get("user:child:age_years")
    diagnosis = tool_context.state.get("user:child:diagnosis")

    if name is None or age_years is None or diagnosis is None:
        return {
            "status": "error",
            "message": "Child profile is missing or incomplete. Please provide name, age, and diagnosis.",
        }

    return {
        "status": "success",
        "name": name,
        "age_years": age_years,
        "diagnosis": diagnosis,
    }


save_child_profile_tool = FunctionTool(func=save_child_profile)
retrieve_child_profile_tool = FunctionTool(func=retrieve_child_profile)

print("Child profile tools initialized.")


Child profile tools initialized.


### Research Tool 

In [3]:
# ==============================
# 2. Research tools (GPT + Claude)
# ==============================

def gpt_child_research_tool(tool_context: ToolContext) -> Dict[str, Any]:
    """
    Use the stored child profile (name, age, diagnosis) to call OpenAI GPT
    and get age- & diagnosis-specific therapy recommendations.
    """
    profile = retrieve_child_profile(tool_context)
    if profile.get("status") != "success":
        return {
            "status": "error",
            "error_message": "Child profile is missing or incomplete. "
                             "Please tell me your child's name, age, and diagnosis first.",
        }

    name = profile["name"]
    age = profile["age_years"]
    diagnosis = profile["diagnosis"]

    prompt = f"""
You are a pediatric developmental specialist.

Child profile:
- Name: {name}
- Age: {age} years
- Diagnosis: {diagnosis}

Tasks:
1. Give a one-sentence summary of {diagnosis} in the context of a {age}-year-old child.
2. Provide recommended therapies tailored SPECIFICALLY for this child's age and condition,
   organized under:
   - language/communication
   - physical/movement
   - occupational/fine motor
   - social/emotional
   - cognitive/learning
3. Phrase everything as supportive, practical advice to the parents of {name}.
"""

    if openai_client is None:
        return {"status": "error", "error_message": "OpenAI client not configured."}

    try:
        result = openai_client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": prompt}],
        )
        content = result.choices[0].message.content
    except Exception as e:
        return {
            "status": "error",
            "error_message": f"GPT call failed: {e}",
        }

    return {
        "status": "success",
        "gpt_research": content,
        "child_profile_used": {
            "name": name,
            "age_years": age,
            "diagnosis": diagnosis,
        },
    }


gpt_child_tool = FunctionTool(func=gpt_child_research_tool)


def claude_child_research_tool(tool_context: ToolContext) -> Dict[str, Any]:
    """
    Use the stored child profile (name, age, diagnosis) to call Claude
    and get age- & diagnosis-specific therapy recommendations.
    """
    profile = retrieve_child_profile(tool_context)
    if profile.get("status") != "success":
        return {
            "status": "error",
            "error_message": (
                "Child profile is missing or incomplete. "
                "Please tell me your child's name, age, and diagnosis first."
            ),
        }

    name = profile["name"]
    age = profile["age_years"]
    diagnosis = profile["diagnosis"]

    prompt = f"""
You are a pediatric developmental specialist.

Child profile:
- Name: {name}
- Age: {age} years
- Diagnosis: {diagnosis}

Tasks:
1. Give a one-sentence summary of {diagnosis} in the context of a {age}-year-old child.
2. Provide recommended therapies tailored SPECIFICALLY for this child's age and condition,
   organized under:
   - language/communication
   - physical/movement
   - occupational/fine motor
   - social/emotional
   - cognitive/learning
3. Phrase everything as supportive, concrete advice to the parents of {name}.
"""

    if anthropic_client is None:
        return {"status": "error", "error_message": "Anthropic client not configured."}

    try:
        response = anthropic_client.messages.create(
            model="claude-3-5-haiku-20241022",
            max_tokens=700,
            messages=[{"role": "user", "content": prompt}],
        )

        text_blocks = [
            block.text
            for block in response.content
            if getattr(block, "type", None) == "text"
        ]
        claude_text = "\n".join(text_blocks).strip()

        if not claude_text:
            return {
                "status": "error",
                "error_message": "Claude returned no text content.",
            }

    except Exception as e:
        return {
            "status": "error",
            "error_message": f"Claude call failed: {e}",
        }

    return {
        "status": "success",
        "claude_research": claude_text,
        "child_profile_used": {
            "name": name,
            "age_years": age,
            "diagnosis": diagnosis,
        },
    }


claude_child_tool = FunctionTool(func=claude_child_research_tool)

print("Research tools initialized (GPT + Claude).")


Research tools initialized (GPT + Claude).


### Research Agents 

In [4]:
# ==============================
# 3. Research agents (LiteLlm)
# ==============================

# Profile agent
profile_agent = LlmAgent(
    name="profile_agent",
    model=LiteLlm(model="openai/gpt-4o-mini"),
    description="Collects and stores child profile info (name, age, diagnosis) in session state.",
    instruction="""
You are a profile manager for a pediatric support system.

Your responsibilities:

1. When the user describes their child (name, age, and diagnosis/condition),
   you MUST call the `save_child_profile` tool to store those details in session state.

2. If the user updates the profile (e.g., new age, additional diagnoses),
   call `save_child_profile` again with the updated information.

3. After saving, briefly confirm what you stored, for example:
   "I've saved Emma's profile: 3 years old with Down syndrome."

4. You MAY use `retrieve_child_profile` when you need to:
   - check what is currently stored, or
   - confirm the existing profile back to the user.

5. You MUST NOT:
   - give therapy recommendations,
   - discuss interventions,
   - or provide developmental advice.
   Your only job is to manage and confirm the stored profile.

All therapy suggestions will be handled by other specialist agents that use this stored profile.
""",
    tools=[save_child_profile_tool, retrieve_child_profile_tool],
)


# GPT researcher agent
gpt_researcher = LlmAgent(
    name="gpt_researcher",
    model=LiteLlm(model="openai/gpt-4o-mini"),
    description="Researcher agent that personalizes therapy insights using GPT via tool.",
    tools=[gpt_child_tool],
    instruction="""
You are a pediatric development research agent.
You MUST always call the tool `gpt_child_tool`.

Do NOT generate your own text.
Do NOT summarize or analyze on your own.

Your ONLY job:
- Call `gpt_child_tool`
- Return the tool output as your final answer.
If the tool returns status="error", explain the error_message to the user and
ask them to provide the child's name, age, and diagnosis.
""",
    output_key="gpt_research",
)


# Claude researcher agent
claude_researcher = LlmAgent(
    name="claude_researcher",
    model=LiteLlm(model="openai/gpt-4o-mini"),
    description="Researcher agent that personalizes therapy insights using Claude via tool.",
    tools=[claude_child_tool],
    instruction="""
You are a pediatric development research agent that delegates all content generation to Claude.

You MUST always call the tool `claude_child_tool`.

Rules:
- Do NOT generate your own domain answer.
- Do NOT summarize or rephrase on your own.
- Do NOT answer directly from your own knowledge.
- Your ONLY job is:
    1) Use the tool (which already uses the child profile from session state),
    2) Return the tool's output as your final answer.
If the tool returns status="error", explain the error_message to the user
and suggest that they provide the child's name, age, and diagnosis.
""",
    output_key="claude_research",
)


# "Gemini-style" researcher, but implemented with LiteLlm
gemini_style_researcher = LlmAgent(
    name="gemini_style_researcher",
    model=LiteLlm(model="openai/gpt-4o-mini"),
    description="Third viewpoint researcher that uses the stored child profile directly.",
    tools=[retrieve_child_profile_tool],
    instruction="""
You are a pediatric developmental specialist and part of a multi-model research team.

REQUIRED behavior:

1. Always call the tool `retrieve_child_profile` to retrieve:
   - child's name
   - child's age (years)
   - child's diagnosis/condition

2. If the tool returns an error (profile missing):
   - Do NOT invent an answer.
   - Say: "I cannot provide recommendations because no child profile is stored yet.
     Please provide your child's name, age, and diagnosis."

3. If the profile exists, produce a personalized research summary:
   - Begin with: "For [NAME], [AGE] years old with [DIAGNOSIS], ..."
   - Provide therapy recommendations tailored to both age & diagnosis.
   - Organize the recommendations under the sections:
        • language & communication
        • physical & movement
        • occupational & fine-motor
        • social & emotional
        • cognitive & learning

4. The tone must be warm, supportive, and parent-focused.
5. Keep the response concise but specific.

Your answer will be stored under the key `gemini_research` and later used by an aggregator.
""",
    output_key="gemini_research",
)

# Parallel research team
parallel_research_team = ParallelAgent(
    name="parallel_research_team",
    sub_agents=[gemini_style_researcher, gpt_researcher, claude_researcher],
)

print("Research agents initialized (all LiteLlm).")


Research agents initialized (all LiteLlm).


### Aggregator and root agents 

In [5]:
# ==============================
# 4. Aggregator & root pipeline
# ==============================

aggregator_agent = LlmAgent(
    name="aggregator_agent",
    model=LiteLlm(model="openai/gpt-4o-mini"),
    tools=[retrieve_child_profile_tool],
    description="Final summarizer that merges multiple research sources into one parent-friendly plan.",
    instruction="""
You are the final summarizer in a multi-model research system.

Upstream agents have already run and stored their outputs in state under these keys
(if they succeeded):

- 'gemini_research'
- 'gpt_research'
- 'claude_research'

You MUST NOT try to reference them as {gemini_research} or similar; instead, you
should conceptually treat them as background sources that the system has already
given you.

Steps:

1. Call `retrieve_child_profile` to get:
   - child name
   - age in years
   - diagnosis

2. If the profile tool returns an error (no child info stored):
   - Do NOT try to summarize.
   - Respond with a short, clear message:
     "I can't summarize therapies yet because no child profile is stored.
      Please tell me your child's name, age, and diagnosis."

3. If the profile exists, produce your answer in EXACTLY this structure:

   Line 1:
     "[NAME] is [AGE] years old and has [DIAGNOSIS]."

   Line 2:
     " [DIAGNOSIS] is ..."
     → one concise sentence explaining the condition in plain language.

   Then a section:

   "Therapy Recommendations by Category"

   Under this, create the following subsections, each with ONE short paragraph
   that summarizes what the three researchers broadly agree on:

   - **Language & Communication**
   - **Physical & Movement**
   - **Occupational & Fine Motor**
   - **Social & Emotional**
   - **Cognitive & Learning**

   Use the information from whatever research keys are present in state
   (gemini_research, gpt_research, claude_research). If one of them is missing,
   just ignore it and rely on the others.

4. After that, add a final section:

   "Differences Between Researchers"

   - If they largely agree overall:
       Write one short sentence stating that all sources are broadly aligned.
   - If there are meaningful differences (e.g., one emphasizes professional services
     more while another emphasizes home play-based work), list 2–4 bullet points
     describing the key differences.

Tone rules:
- Warm, supportive, and parent-friendly.
- Concise but specific.
- No long bullet lists; keep it readable.
""",
    output_key="final_summary",
)

root_agent = SequentialAgent(
    name="research_system",
    sub_agents=[
        profile_agent,          # store/update profile
        parallel_research_team, # run three researchers in parallel
        aggregator_agent,       # final summary
    ],
)


### Runner and Helper

In [6]:
# ==============================
# 5. App, runner, helper
# ==============================

APP_NAME = "genex_research_app"
USER_ID = "emma_parents"

research_app = App(
    name=APP_NAME,
    root_agent=root_agent,
)

runner = Runner(
    app=research_app,
    session_service=session_service,
)

print("Genex research app initialized.")
print(f"  App name: {APP_NAME}")


async def ask(prompt: str):
    """
    Convenience helper: send a prompt into the research app and print the final summary if present.
    """
    events = await runner.run_debug(
        prompt,
        user_id=USER_ID,
        session_id="emma-session-001",
        quiet=False,
        verbose=True,
    )

    final_event = events[-1]
    summary = None
    if hasattr(final_event.actions, "state_delta"):
        summary = final_event.actions.state_delta.get("final_summary")

    if summary:
        print("\nFINAL SUMMARY\n")
        print(summary)
    else:
        print("\n(No final_summary in last event. Check logs above.)")


App name mismatch detected. The runner is configured with app name "genex_research_app", but the root agent was loaded from "C:\Users\T490\Downloads\Genetics-Dashboard\.venv\Lib\site-packages\google\adk\agents", which implies app name "agents".


Genex research app initialized.
  App name: genex_research_app


In [7]:
await ask("My child's name is Emma. She is 3 years old and has Down syndrome.")



 ### Created new session: emma-session-001

User > My child's name is Emma. She is 3 years old and has Down syndrome.
profile_agent > [Calling tool: save_child_profile({'name': 'Emma', 'age_years': 3, 'diagnosis': 'Dow...)]
profile_agent > [Tool result: {'status': 'success', 'message': 'Stored child profile: Emma, 3 years old, diagnosis: Down syndrome....]
profile_agent > I've saved Emma's profile: 3 years old with Down syndrome.
gpt_researcher > [Calling tool: gpt_child_research_tool({})]
gpt_researcher > [Tool result: {'status': 'success', 'gpt_research': "### 1. Summary of Down Syndrome\nDown syndrome is a genetic c...]
gemini_style_researcher > [Calling tool: retrieve_child_profile({})]
claude_researcher > [Calling tool: claude_child_research_tool({})]
gemini_style_researcher > [Tool result: {'status': 'success', 'name': 'Emma', 'age_years': 3, 'diagnosis': 'Down syndrome'}]
claude_researcher > [Tool result: {'status': 'success', 'claude_research': "1. Down Syndrome Overview:\nDow

In [8]:
await ask("My child's name is Emma. She is 3 years old and has Down syndrome.")


 ### Continue session: emma-session-001

User > My child's name is Emma. She is 3 years old and has Down syndrome.
profile_agent > [Calling tool: save_child_profile({'name': 'Emma', 'age_years': 3, 'diagnosis': 'Dow...)]
profile_agent > [Tool result: {'status': 'success', 'message': 'Stored child profile: Emma, 3 years old, diagnosis: Down syndrome....]
profile_agent > I've saved Emma's profile: 3 years old with Down syndrome.
gemini_style_researcher > [Calling tool: retrieve_child_profile({})]
claude_researcher > [Calling tool: claude_child_research_tool({})]
gemini_style_researcher > [Tool result: {'status': 'success', 'name': 'Emma', 'age_years': 3, 'diagnosis': 'Down syndrome'}]
claude_researcher > [Tool result: {'status': 'success', 'claude_research': "Child Profile Assessment for Emma\n\n1. Down Syndrome Over...]
gpt_researcher > [Calling tool: gpt_child_research_tool({})]
gpt_researcher > [Tool result: {'status': 'success', 'gpt_research': '### One-Sentence Summary\nDown syndro

In [9]:
await ask("What therapies do you recommend for Emma's overall development?")


 ### Continue session: emma-session-001

User > What therapies do you recommend for Emma's overall development?
profile_agent > I'm unable to provide therapy recommendations directly. However, I can help you manage and confirm Emma's profile details. If you have any specific updates or additional information about Emma that you would like to store, please let me know!
gemini_style_researcher > [Calling tool: retrieve_child_profile({})]
gemini_style_researcher > [Tool result: {'status': 'success', 'name': 'Emma', 'age_years': 3, 'diagnosis': 'Down syndrome'}]
gpt_researcher > [Calling tool: gpt_child_research_tool({})]
gpt_researcher > [Tool result: {'status': 'success', 'gpt_research': "### Summary of Down Syndrome\nDown syndrome is a genetic cond...]
claude_researcher > [Calling tool: claude_child_research_tool({})]
gemini_style_researcher > For Emma, who is 3 years old and has Down syndrome, here are some personalized therapy recommendations for her overall development:

### Languag

In [10]:
await ask("Can you suggest more fine-motor activities for Emma?")


 ### Continue session: emma-session-001

User > Can you suggest more fine-motor activities for Emma?
profile_agent > I'm unable to provide specific activity suggestions directly. However, if you'd like to update or manage Emma's profile with new information or changes, please let me know!
gemini_style_researcher > [Calling tool: retrieve_child_profile({})]
gemini_style_researcher > [Tool result: {'status': 'success', 'name': 'Emma', 'age_years': 3, 'diagnosis': 'Down syndrome'}]
claude_researcher > [Calling tool: claude_child_research_tool({})]
claude_researcher > [Tool result: {'status': 'success', 'claude_research': "Here's a comprehensive developmental guidance plan for Emm...]
gpt_researcher > [Calling tool: gpt_child_research_tool({})]
gemini_style_researcher > For Emma, who is 3 years old and has Down syndrome, here are some additional fine-motor activities that can support her development:

1. **Playdough Fun**: Encourage her to roll, squish, and shape playdough, which helps st